In [10]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

In [3]:
train_data = pd.read_csv('../res/train_dataset.csv')
test_data = pd.read_csv('../res/test_dataset.csv')

In [4]:
train_data.head()

,sha256,md5,label,numstrings,printables,entropy,paths,urls,registry,MZ,...,subsystem,major_image_version,minor_image_version,major_operating_system_version,minor_operating_system_version,sizeof_code,sizeof_headers,entry,imports,exports
0,e42b75fa32c93b75d370d61c947098cabd8bf1dd55a5ee...,e2dbe348a6370f586fab7e9568ccdabe,0,331,7043,5.334609,2,20,0,2,...,WINDOWS_CUI,0,0,4,0,9728,512,.text,mscr__CrDllMn,NaN
1,2fc6d5fc12addf0259afe57b22e510318a84274a1e8680...,ee0f8c6df3cc07c6c3d35ec5b32ffe8b,1,433,3215,6.469070,0,0,0,4,...,WINDOWS_GUI,0,0,5,0,0,4096,.text,CMDLG32_PgStpDlgW|ntdll_mmcpy|ntdll_mmst|ntdll...,NaN
2,10792606fc063281ff2ae288cdbdb77675fd10777278ec...,614dce4e17952b1f886848021bd56645,0,308,6165,5.618080,0,26,0,2,...,WINDOWS_GUI,0,0,5,0,19456,1024,.text,MSVCR90_strchr|MSVCR90_strcmp|MSVCR90__vsnprnt...,?fnJsonDll@@YAHXZ|json_object_array_add|json_o...
3,0bfb3803ee117aed607150084201cf5626257a2fc8bae6...,5fef689041958f6b08550e5a8a569048,1,806,6130,5.740129,0,0,0,3,...,WINDOWS_GUI,0,0,5,0,15360,1024,.text,PWRPRF_GtCrrntPwrPlcs|SR32_GtCrsrnf|SR32_PstMs...,NaN
4,004575d622dfdb7e2904d5b182089ab3c9fa7ce193746d...,b6dc84da891d6b2360d996244626a436,1,5818,1399909,5.011869,0,9,0,11,...,WINDOWS_GUI,0,0,4,0,443392,1024,CODE,krnl32_DltCrtclSctn|krnl32_TlsStVl|krnl32_lstr...,NaN


In [29]:
X = train_data.drop(['sha256', 'md5', 'label', 'subsystem'], axis=1)
y = train_data['label']

X.fillna(0, inplace=True)

numeric_cols = X.select_dtypes(include=['number']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

X[categorical_cols] = X[categorical_cols].astype(str)

label_encoder = LabelEncoder()

for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])


scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

In [13]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

RandomForestClassifier(random_state=42)

In [16]:
X_test = test_data.drop(['sha256', 'md5', 'subsystem'], axis=1)

X_test.fillna(0, inplace=True)

numeric_cols_test = X_test.select_dtypes(include=['number']).columns
categorical_cols_test = X_test.select_dtypes(include=['object']).columns

X_test[categorical_cols_test] = X_test[categorical_cols_test].astype(str)

for col in categorical_cols_test:
    X_test[col] = label_encoder.fit_transform(X_test[col])

X_test[numeric_cols_test] = scaler.transform(X_test[numeric_cols_test])

predictions = model.predict(X_test)

In [18]:
results = pd.DataFrame({'ID': test_data['sha256'], 'label': predictions})

results.to_csv('predictions.csv', index=False)

In [25]:
# Charger le fichier "sample.csv" qui contient les ID corrects
sample_data = pd.read_csv('../res/sample.csv')

# Charger le fichier "predictions.csv"
results = pd.read_csv('predictions.csv')

# Remplacer la colonne "id" dans les résultats par celle du fichier "sample.csv"
results['ID'] = sample_data['ID']

# Sauvegarder les résultats finaux avec les ID corrects au format CSV
results.to_csv('predictions.csv', index=False)


In [26]:
results.head()

,ID,label
0,510000,1
1,510001,0
2,510002,1
3,510003,0
4,510004,1
